<a href="https://colab.research.google.com/github/samibahig/RecoverProject/blob/main/RandomSCMProteomique_Cyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
### Ici on download les données, on les nettoie et on les met sous forme de matrices: proteomics pour les features et metadata pour les labels, 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
#print('\nMETADATA :')
data_path = ''
metadata_filename = '/content/metadata.csv'
meta_df = pd.read_csv(metadata_filename)
print('-----------')
print(meta_df)
print('-----------')
#print(meta_df.columns)
meta_df.columns = ['#', 'plate', '-', 'symptoms'] + list(meta_df)[4:]
#print(meta_df.columns)
#print('available metadata :', list(meta_df))
meta_idx = meta_df['ID'].to_list()
meta_label = meta_df['symptoms'].to_list()
#print('------------------')
#print(list(zip(meta_idx, meta_label)))
#print('------------------')
meta_id_label_dict = {str(k): 1 if v=='S' else 0 for k, v in zip(meta_idx, meta_label)}

#DF1 : proteomics
#print('\nPROEOMICS DATA :')
proteomics_data_filename = '/content/proteomics_cyt.csv'

dim_df = pd.read_csv(proteomics_data_filename, nrows=1)
#print('--------')
#print(dim_df)
#print('--------')
dim = len(list(dim_df))
#print(dim)
#print('------------')
print('# of columns in source csv file :', dim)
all_cols = [i for i in range(dim)]

print('--------')
feat_cols = all_cols[1:-4]
print(feat_cols)
print('--------')
samplesidx_col = [0]

feat_df = pd.read_csv(proteomics_data_filename, skiprows=4, nrows=1, dtype=str, usecols=feat_cols)
features = list(feat_df)
print('# of features : ', len(features))
print('first feature :', features[0])
print('last feature :', features[-1])

idx_df = pd.read_csv(proteomics_data_filename, skiprows=6, index_col=0, skipfooter=4, usecols=[0], engine='python')
idx = list(idx_df.index.values)
print('# of idx : ', len(idx))
print('first id :', idx[0])
print('last id :', idx[-1])

df1 = pd.read_csv(proteomics_data_filename, skiprows=6, skipfooter=4, usecols=feat_cols, engine='python')
df1.fillna(value=0, axis=1, inplace=True, limit=None)
assert df1.shape[0] == len(idx)
assert df1.shape[1] == len(features)

df1['idx'] = idx
df1.set_index('idx', inplace=True)
df1.columns = features
print('# of Nan values :', df1.isna().sum().sum())

#clean data of samples that are not in metadata :
idx = df1.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df1['label'] = y
df1 = df1[df1.label != 'to_remove']

#create X and y matrices for ML :
y = list(df1['label'])
del df1['label']
print('---------')
df1.drop(columns=['P01133'], inplace=True)
#df1.drop('3-009', inplace=True)
print(df1)
print('---------')
X = df1.to_numpy()
print('proteomics_cyst data :')
print('# of samples : ', df1.shape[0])
print('# of features : ', df1.shape[1])
print('labels:', list(dict.fromkeys(y)))


## save X and y in pickles if you want :
##data_name = 'recover_multiomics_'
##feat_dict = {k: str(v) for k, v in zip(range(len(list(df))), list(df))}
##with open(data_path + data_name + 'feat_dict', 'wb') as fo:
##    pkl.dump(feat_dict, fo)
##with open(data_path + data_name + 'X', 'wb') as fo:df
##            pkl.dump(X, fo)
##with open(data_path + data_name + 'y', 'wb') as fo:
##            pkl.dump(y, fo)


-----------
    Unnamed: 0 Unnamed: 1  ... addition                                specify.1
0            1    Plate 1  ...      NaN                                      NaN
1            2    Plate 1  ...      5.0                                   Stress
2            3    Plate 1  ...      6.0  back pain is worse, shortness of breath
3            4    Plate 1  ...      2.0  Impression of a foreingh body in throat
4            5    Plate 1  ...      2.0                                      NaN
..         ...        ...  ...      ...                                      ...
95         100    Plate 2  ...      NaN                                      NaN
96         101    Plate 2  ...      NaN                                      NaN
97         102    Plate 2  ...      NaN                                      NaN
98         103    Plate 2  ...      NaN                                      NaN
99         104    Plate 2  ...      NaN                                      NaN

[100 rows x 46 

In [84]:
#df1.tail(20)
#df1['P01133']
#
#df1.drop([df1['P01133'] == '> ULOQ'], inplace = True)

In [92]:
df1.columns

Index(['P80075', 'O95760', 'P48061', 'P78380', 'Q8NEV9_Q14213', 'P60568',
       'Q07325', 'P01135', 'P01584', 'P05231', 'P05112', 'O43508', 'Q969D9',
       'P51671', 'P14210', 'P49771', 'Q96PD4', 'P13232', 'P35225', 'Q14116',
       'Q99616', 'P50591', 'P02778', 'P01579', 'P22301', 'Q99731', 'P01375',
       'P40933', 'P10147', 'P10145', 'P39900', 'P04141', 'P09919', 'P15692',
       'Q9P0M4', 'P13500', 'Q16552', 'P13725', 'P09603', 'P13236', 'O14625',
       'P01374'],
      dtype='object')

In [89]:
df1.fillna(value=0, axis=1, inplace=True, limit=None)
df1.head(50)
#df1 = df1.drop(labels='> ULOQ', axis=1, inplace=True)
df1

,pg/mL,pg/mL.1,pg/mL.2,pg/mL.3,pg/mL.4,pg/mL.5,pg/mL.6,pg/mL.7,pg/mL.8,pg/mL.9,pg/mL.10,pg/mL.11,pg/mL.12,pg/mL.13,pg/mL.14,pg/mL.15,pg/mL.16,pg/mL.17,pg/mL.18,pg/mL.19,pg/mL.20,pg/mL.21,pg/mL.22,pg/mL.23,pg/mL.24,pg/mL.25,pg/mL.26,pg/mL.27,pg/mL.28,pg/mL.29,pg/mL.30,pg/mL.31,pg/mL.32,pg/mL.33,pg/mL.34,pg/mL.35,pg/mL.36,pg/mL.37,pg/mL.38,pg/mL.39,pg/mL.40,pg/mL.41,pg/mL.42
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,32.88848,0.55984,397.29429,146.91341,1.49025,0.0031,47.63594,19.60223,0.01838,4.08696,0.02049,841.61841,0.02052,120.03389,655.88138,180.05744,0.04859,5.54881,20.82823,263.5005,122.60355,367.37495,107.92546,0.30287,1.3686,130.27204,19.46435,11.99858,7.40087,10.74015,185.29854,0.28222,84.90633,460.05592,7.50828,440.98377,546.78923,0.18685,11.57456,169.21768,91.57302,45.7468,18.96545
2,81.61041,0.06288,340.58883,513.65527,2.62802,0.01274,20.77929,36.61691,0.13591,7.68976,0.0522,685.44529,0.02987,120.02688,1072.31251,119.73066,0.47567,4.10289,0.29985,246.075,186.67814,396.55929,51.91873,0.12844,0.59468,99.51079,10.33548,10.15531,6.6197,11.14864,154.31861,0.15098,97.96295,427.58919,12.9628,490.3056,466.59691,0.15477,15.16106,133.40028,195.10538,35.51699,11.14506
3,21.05386,0.21039,269.3803,195.82065,6.40496,0.02725,59.49787,19.51414,0.13468,3.96441,0.04595,824.84789,0.03021,98.03313,630.98303,180.63118,3.59341,4.22493,0.30544,197.57757,165.57297,611.68764,91.07628,0.23817,0.74935,114.361,21.53479,11.63106,8.86735,9.76674,221.43462,0.24339,46.79207,540.76521,32.22662,572.97188,528.97433,0.71538,7.18436,130.32299,307.71337,45.26911,7.55923
4,65.32818,0.17781,213.10409,221.91738,3.62822,0.02938,87.22272,8.42985,0.03051,5.64413,0.01686,704.85085,0.01658,170.40823,466.69849,115.24552,4.90308,6.883,0.63191,259.29947,194.00077,774.57014,172.86498,0.52656,0.95139,174.72671,21.80639,13.05225,15.51203,12.51854,358.06817,0.26974,85.96804,502.53571,8.03254,321.30365,609.9247,0.22472,3.80081,138.49611,133.96489,62.91014,14.23622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.029802322,0.238418579,30.51757809,0.953674317,0.238418579,0.11920929,0.11920929,0.476837158,0.190734863,0.059604645,0.059604645,3.814697264,0.476837158,0.238418579,0.238418579,0.953674318,0.461511623,0.238418579,0.476837158,0.953674317,0.059604645,0.953674317,0.11920929,0.029802322,0.21826022,0.953674318,3.814697267,0.11920929,0.029802322,0.238418579,7.629394533,0.238418579,7.62939454,0.476837158,1.90734863,0.476837158,0.238418579,0.11920929,0.11920929,0.238418579,0.11920929,0.11920929,0.11920929
116,0.029802322,0.238418579,30.51757809,0.953674317,0.238418579,0.11920929,0.11920929,0.476837158,0.190734863,0.059604645,0.059604645,3.814697264,0.476837158,0.238418579,0.238418579,0.953674318,0.336278806,0.238418579,0.476837158,0.953674317,0.059604645,0.953674317,0.11920929,0.029802322,0.164854579,0.953674318,3.814697267,0.11920929,0.029802322,0.238418579,7.629394533,0.238418579,7.62939454,0.476837158,1.90734863,0.476837158,0.238418579,0.11920929,0.11920929,0.238418579,0.11920929,0.11920929,0.11920929
117,0.00654322,0.090658947,22.5492884,0,0.039103235,0.003278328,0,0.380296159,0.035011025,0.00623886,0.018508553,0,0.080167425,0.055941269,0.130523906,0.04623328,0.681152304,0.151134977,0.042896645,0.088227115,0.003589045,0.002936432,0,0.008157265,0.368959649,0.368654726,1.793279401,0.154110138,0.011923965,0.034521,0,0,5.678050358,0.010908219,0.633173316,0.048226292,0.015506466,0.051797935,0,0.07180584,0,0.038301252,0.120293123
118,0.003132865,0.031866869,13.10544896,0,0.045446061,0.003339611,0,0.362257312,0.056722801,0.004342629,0.018912309,0,0.159709352,0.063818236,0.056850042,0,0.461511623,0.143669181,0.03346799,0.090372695,0,0,0,0.011990984,0.21826022,0.329804173,2.531876262,0.073572051,0.006164947,0.033517107,0,0,2.337840008,0.009423031,0.639602008,0.092627351,0,0.061460904,0,0.057046267,

In [56]:
df1.to_csv("/content/df1.csv")
df1

,1.62496,1.67346,2.30191,1.09831,0.81814,6.64685,4.33614,2.09165,3.05241,2.20853,2.26132,2.50169,0.23786,4.26749,0.64027,3.49041,1.65639,0.18282,-0.00258,4.99097,2.04746,0.91502,3.26891,2.64068,3.40662,1.75302,0.12112,1.8423,2.36297,2.11362,1.17813,3.05439,1.92376,9.62026,8.21851,2.76462,1.76014,2.25881,2.59258,-0.81823,...,0.17781,213.10409,221.91738,3.62822,0.02938,87.22272,8.42985,0.03051,5.64413,0.01686,704.85085,0.01658,170.40823,466.69849,115.24552,4.90308,6.883,0.63191,259.29947,194.00077,774.57014,172.86498,0.52656,0.95139,174.72671,21.80639,13.05225,15.51203,12.51854,358.06817,0.26974,85.96804,502.53571,8.03254,321.30365,609.9247,0.22472,3.80081,138.49611,133.96489
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1-026,2.48623,2.42240,2.86282,2.19087,1.65184,5.22605,6.28107,1.36379,3.06955,2.27634,2.07234,2.20169,0.42551,3.32472,1.34123,3.66986,1.94632,0.19312,0.26522,5.33663,1.55030,1.35359,4.92914,1.57654,3.53306,2.05642,0.09155,2.80670,2.72684,2.45478,1.10694,3.07896,1.96822,8.34351,8.50506,2.96392,3.36216,0.08286,1.90096,-0.39492,...,0.20685,303.32904,538.27425,8.17765,0.02086,71.46363,26.32494,0.10661,3.74722,0.01831,671.56909,0.14182,216.81600,758.27687,46.06363,0.43898,10.52915,0.07257,183.78762,133.55088,661.44866,120.35070,0.29943,16.04506,44.84258,91.40858,12.10870,8.32337,7.81655,96.37740,0.09573,59.29720,486.41575,12.10414,520.15895,498.29936,1.25437,14.47713,122.28971,202.38365
5-160,2.51644,1.42410,2.66267,1.01372,1.29349,6.88173,5.36133,1.76990,2.91154,1.49769,2.99616,2.28051,0.48042,3.31877,1.08041,3.35323,2.07311,-0.36846,1.32608,4.59227,1.58213,0.95043,4.03325,1.52062,2.37259,2.10056,0.64504,2.14644,2.11373,3.16644,1.47408,3.79823,2.95064,7.91852,8.24384,3.23319,2.34342,0.09271,2.51827,-0.24256,...,0.10408,358.65048,340.85689,6.99217,0.01757,93.01169,13.04904,0.08250,4.61720,0.03077,1068.27309,0.03470,154.59001,617.55142,170.40326,0.52644,10.59590,0.30125,317.73481,294.00150,812.09222,241.02610,0.08465,1.11067,134.06110,22.78650,18.97588,25.67310,23.13243,293.96144,0.15127,68.91151,774.71499,7.07773,254.5813,789.30173,0.09381,4.41604,139.86582,237.97159
1-057,2.18885,1.92321,2.39325,0.88833,1.09321,6.04054,5.61824,1.48480,2.63778,1.89899,2.24617,2.51717,0.22299,3.55706,1.45571,3.38502,1.01922,-0.12618,-0.16629,4.67541,1.99160,1.23945,3.79489,1.43101,3.61266,1.66163,-0.16582,2.32807,2.26160,2.96670,2.96144,2.31760,1.77681,7.87806,7.78202,2.28510,2.74136,0.19890,1.83341,-0.65013,...,0.12938,253.87535,421.66801,2.64339,0.01609,44.71970,18.09288,0.13899,5.61282,0.03867,919.53078,0.10944,114.11861,651.62065,111.86342,1.35037,6.24202,0.68117,176.17564,175.41600,565.65771,160.38016,0.28778,1.32927,125.85381,20.58934,15.22033,6.56485,5.91469,121.55361,0.65329,71.92847,1002.93908,16.51496,855.03256,354.65773,0.88035,9.17426,125.66622,108.02697
2-026,0.95533,1.76364,1.51776,0.58076,0.69636,5.90082,2.73350,2.19529,3.23191,0.98465,3.04368,1.81168,0.40645,4.67359,0.77899,3.54338,0.86176,-0.30449,0.33032,3.76795,1.80045,1.22304,2.54004,1.40856,2.94573,1.82917,-0.04098,1.54045,1.94063,1.98781,1.19970,2.33297,2.39776,8.11420,7.38280,3.21477,1.33949,0.16812,2.10073,-0.09575,...,0.08390,288.98831,179.43192,3.47052,0.03396,54.19521,17.44564,0.05448,2.32773,0.05600,628.13830,1.58528,78.84977,461.00918,140.69730,0.51683,6.11277,1.46852,184.06353,84.07957,225.18583,144.84396,0.24226,0.70483,174.41203,17.54087,10.35005,4.45898,11.75630,133.80343,0.15385,37.76285,719.29010,6.70568,541.45392,406.54303,0.06454,6.68895,162.27642,90.57557
1-098,2.89161,1.11551,3.99761,1.85142,1.87377,5.69155,5.97976,1.76262,2.96242,2.02689,2.34766,3.04282,0.86051,5.42346,1.99981,3.53259,2.11203,0.17115,0.56939,5.09035,2.27526,1.96526,4.92574,1.84684,2.97131,2.68184,0.65976,2.99761,2.34217,2.96187,0.72550,2.73521,2.62436,8.12573,8.07679,3.35168,3.01710,1.26085,1.82416,1.40533,...,0.41453,435.77899,436.55285,4.33455,0.01037,48.68184,29.89121,0.17327,8.39665,0.02531,812.44530,0.24094,134.35920,863.40983,66.57326

In [42]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import numpy as np
scaler = preprocessing.MinMaxScaler()
scaler.fit(X)
#print(normalized_arr)
#scaler

ValueError: ignored

In [93]:
pip install pyscm-ml   ### Ici on installe pyscm-ml librairie indispensable pour rouler l'algorithme RandomSCM

In [94]:
### Ici c'est l'algorithme SCM avec ses classes et toutes ses fonctions
from sklearn.base import ClassifierMixin
from sklearn.ensemble import BaseEnsemble
from pyscm import SetCoveringMachineClassifier
from pyscm.rules import DecisionStump

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state
from sklearn.utils.random import sample_without_replacement
from sklearn.metrics import accuracy_score
import numbers
import itertools
import numpy as np
from warnings import warn
from joblib import effective_n_jobs, Parallel, delayed

MAX_INT = np.iinfo(np.int32).max
def _parallel_build_estimators(idx, ensemble, p_of_estims, seeds, X, y, tiebreaker):
    """
    Fit SCM estimators on subsamples of the training data
    """
    estimators = []
    estim_features = []
    for k in idx:
        p_param = p_of_estims[k] # p param for the classifier to fit
        random_state = seeds[k]
        estim = SetCoveringMachineClassifier(p=p_param, max_rules=ensemble.max_rules, model_type=ensemble.model_type, random_state=random_state)
        feature_indices = sample_without_replacement(ensemble._pop_features, ensemble._max_features, random_state=random_state)
        samples_indices = sample_without_replacement(ensemble._pop_samples, ensemble._max_samples, random_state=random_state)
        Xk = (X[samples_indices])[:, feature_indices]
        yk = y[samples_indices]
        if len(list(set(yk))) < 2:
            raise ValueError("One of the subsamples contains elements from only 1 class, try increase max_samples value")
        if tiebreaker is None:
            estim.fit(Xk, yk)
        else:
            estim.fit(Xk, yk, tiebreaker)
        estim_features.append(feature_indices)
        estimators.append(estim)
    return estimators, estim_features


def _parallel_predict_proba(ensemble, X, idx, results):
    """
    Compute predictions of SCM estimators
    """
    for k in idx:
        res = ensemble.estimators[k].predict(X[:, ensemble.estim_features[k]])
        if ensemble.min_cq_combination:
            res[res==0] = -1
            results = results + ensemble.min_cq_weights[k]*res
        else:
            results = results + res
    return results


def _partition_estimators(n_estimators, n_jobs):
    """Private function used to partition estimators between jobs."""
    # Compute the number of jobs
    n_jobs = min(effective_n_jobs(n_jobs), n_estimators)

    # Partition estimators between jobs
    n_estimators_per_job = np.full(n_jobs, n_estimators // n_jobs,
                                   dtype=np.int)
    n_estimators_per_job[:n_estimators % n_jobs] += 1
    starts = np.cumsum(n_estimators_per_job)

    return n_jobs, n_estimators_per_job.tolist(), [0] + starts.tolist()


class RandomScmClassifier(BaseEnsemble, ClassifierMixin):
    """A Bagging classifier for SetCoveringMachineClassifier()
    The base estimators are built on subsets of both samples
    and features.
    Parameters
    ----------
    n_estimators : int, default=10
        The number of base estimators in the ensemble.
    max_samples : int or float, default=1.0
        The number of samples to draw from X to train each base estimator with
        replacement.
        - If int, then draw `max_samples` samples.
        - If float, then draw `max_samples * X.shape[0]` samples.
    max_features : int or float, default=1.0
        The number of features to draw from X to train each base estimator (
        without replacement.
        - If int, then draw `max_features` features.
        - If float, then draw `max_features * X.shape[1]` features.
    max_rules : int
        maximal number of rules for the scm estimators
    p_options : list of float with len =< n_estimators, default=[1.0]
        The estimators will be fitted with values of p found in p_options
        let k be k = n_estimators/len(p_options),
        the k first estimators will have p=p_options[0],
        the next k estimators will have p=p_options[1] and so on...
    model_type : string, default='conjunction'
        type of estimators to build
        accepted values : 'conjunction', 'disjunction'
    n_jobs : int, default=None
        The number of jobs to run in parallel for both fit and
        predict. 'None' means 1. '-1' means using all
        processors.
    random_state : int or RandomState, default=None
        Controls the random resampling of the original dataset
        (sample wise and feature wise).
        If the base estimator accepts a `random_state` attribute, a different
        seed is generated for each instance in the ensemble.
        Pass an int for reproducible output across multiple function calls.
        See :term:`Glossary <random_state>`.

    Attributes
    ----------
    n_features_ : int
        The number of features when :meth:`fit` is performed.
    estimators_ : list of estimators
        The collection of fitted base estimators.
    estim_features : list of arrays
        The subset of drawn features for each base estimator.

    Examples
    --------
    >>> random_scm = RandomScmClassifier(p_options=[2, 4], max_samples=0.5, max_features = 0.7)
    >>> random_scm.fit(X_train, y_train)
    >>> hyperparams = random_scm.get_hyperparams()
    >>> importances = random_scm.features_importance()
    >>> disagree = random_scm.classifiers_disagreement(X)

    References
    ----------
    .. [1] L. Breiman, "Pasting small votes for classification in large
           databases and on-line", Machine Learning, 36(1), 85-103, 1999.
    .. [2] G. Louppe and P. Geurts, "Ensembles on Random Patches", Machine
           Learning and Knowledge Discovery in Databases, 346-361, 2012.
    """
    def __init__(self,
                 n_estimators=100,
                 max_samples=0.5,
                 max_features=0.5,
                 max_rules=10,
                 p_options=[1.0],
                 model_type="conjunction",
                 n_jobs=None,
                 min_cq_combination=False,
                 min_cq_mu = 10e-3,
                 random_state=None):
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_features = max_features
        self.max_rules = max_rules
        self.p_options = p_options
        self.model_type = model_type
        self.n_jobs = n_jobs
        self.min_cq_combination = min_cq_combination
        self.min_cq_mu = min_cq_mu
        self.random_state = random_state
        self.labels_to_binary = {}
        self.binary_to_labels = {}

    def p_for_estimators(self):
        """Return the value of p for each estimator to fit."""
        options_len = len(self.p_options) # number of options
        estims_with_same_p = self.n_estimators // options_len # nb of estimators to fit with the same p
        p_of_estims = []
        if options_len > 1:
            for k in range(options_len - 1):
                opt = self.p_options[k] # an option
                p_of_estims = p_of_estims + ([opt] * estims_with_same_p) # estims_with_same_p estimators with p=opt
        p_of_estims = p_of_estims + ([self.p_options[-1]] * (self.n_estimators - len(p_of_estims)))
        return p_of_estims

    def get_estimators(self):
        """Return the list of estimators of the classifier"""
        if hasattr(self, 'estimators'):
            return self.estimators
        else:
            return "not defined (model not fitted)"

    def get_hyperparams(self):
        """Return the model hyperparameters"""
        hyperparams = {
            'n_estimators' : self.n_estimators, 
            'max_samples' : self.max_samples, 
            'max_features' : self.max_features, 
            'max_rules' : self.max_rules, 
            'p_options' : self.p_options, 
            'model_type' : self.model_type, 
            'random_state' : self.random_state
        }
        return hyperparams

    def labels_conversion(self, labels_list):
        """
        Return the equivalence between labels and binaries
        """
        l = list(set(labels_list))
        labels_dict = {c:idx for idx, c in enumerate(l)}
        if len(l) < 2:
            raise ValueError("Only 1 classe given to the model, needs 2")
        elif len(l) > 2:
             raise ValueError("{} classes were given, multiclass prediction is not implemented".format(len(l)))
        return np.array(l), labels_dict


    def fit(self, X, y, get_feature_importances=True, tiebreaker=None):
        """
        Fit the model with the given data
        """
        # Check if 2 classes are inputed and convert labels to binary labels
        X, y = check_X_y(X, y)
        self.classes_, self.labels_to_binary = self.labels_conversion(y)
        self.binary_to_labels = {bin_label:str_label for str_label, bin_label in self.labels_to_binary.items()}
        y = np.array([self.labels_to_binary[l] for l in y])

        # Save the original number of features
        self.n_features = X.shape[1]

        self.estimators = []
        self.estim_features = []
        max_rules = self.max_rules
        p_of_estims_ = self.p_for_estimators()
        model_type = self.model_type

        #seeds for reproductibility
        random_state = self.random_state
        random_state = check_random_state(random_state)
        seeds = random_state.randint(MAX_INT, size=self.n_estimators)
        self._seeds = seeds

        pop_samples, pop_features = X.shape
        max_samples, max_features = self.max_samples, self.max_features

        # validate max_samples
        if not isinstance(max_samples, numbers.Integral):
            max_samples = int(max_samples * pop_samples)
        if not (0 < max_samples <= pop_samples):
            raise ValueError("max_samples must be in (0, n_samples)")
        # store validated integer row sampling values
        self._max_samples = max_samples
        self._pop_samples = pop_samples

        # validate max_features
        if isinstance(self.max_features, numbers.Integral):
            max_features = self.max_features
        elif isinstance(self.max_features, np.float):
            max_features = self.max_features * pop_features
        else:
            raise ValueError("max_features must be int or float")
        if not (0 < max_features <= pop_features):
            raise ValueError("max_features must be in (0, n_features)")
        max_features = max(1, int(max_features))
        # store validated integer feature sampling values
        self._max_features = max_features
        self._pop_features = pop_features

        # parallel loop
        n_jobs, n_estimators_list, starts = _partition_estimators(self.n_estimators, self.n_jobs)
        # building estimators
        all_results = Parallel(n_jobs=n_jobs)(delayed(_parallel_build_estimators)(
                range(starts[i],starts[i+1]), self, p_of_estims_, seeds, X, y, tiebreaker)
            for i in range(n_jobs))

        self.estimators += list(itertools.chain.from_iterable(
            t[0] for t in all_results))
        self.estim_features += list(itertools.chain.from_iterable(
            t[1] for t in all_results))

        if self.min_cq_combination:
            predictions = np.zeros((X.shape[0], self.n_estimators))
            for k in range(self.n_estimators):
                predictions[:, k] = self.estimators[k].predict(X[:, self.estim_features[k]])

            from .min_cq import MinCqLearner
            mincq = MinCqLearner(self.min_cq_mu, "stumps", n_stumps_per_attribute=1,
                                 self_complemented=False)
            mincq.fit(predictions, y)
            self.min_cq_weights = mincq.majority_vote.weights
            self.min_cq_weights /= np.sum(np.abs(self.min_cq_weights))

        if get_feature_importances:
            importances = self.features_importance()
            self.feature_importances_ = np.array([importances['avg'][k]
                                                  if k in importances['avg'] else 0
                                                  for k in range(self.n_features)])
            self.feature_importances_ /= np.sum(self.feature_importances_)

    def predict(self, X):
        """
        Compute model predictions for data in X

        Returns:
        ----------
        predictions : array
            predictions[i] is the predicted class for he sample i in X
        """
        check_is_fitted(self, ["estimators", "estim_features"])
        X = check_array(X)
        predicted_proba = self.predict_proba(X)
        predictions = np.array(np.argmax(predicted_proba, axis=1), dtype=int)
        predictions = np.array([self.binary_to_labels[l] for l in predictions])
        return predictions

    def predict_proba(self, X):
        """
        Predict class probabilities according to the model estimators

        Parameters :
        ----------
        X : array
            a dataset to predict

        Returns : 
        ----------
        proba : array
            proba[c] contains the estimated probability for each sample of X to belong to class c
        """
        check_is_fitted(self, ["estimators", "estim_features"])
        X = check_array(X)
        # parallel loop
        n_jobs, n_estimators_list, starts = _partition_estimators(self.n_estimators, self.n_jobs)
        results = np.zeros(X.shape[0])
        results = Parallel(n_jobs=n_jobs)(delayed(_parallel_predict_proba)(
                self, X, range(starts[i], starts[i+1]), results)
                for i in range(n_jobs))
        if not self.min_cq_combination:
            votes = sum(results) / self.n_estimators
            proba = np.array([np.array([1 - vote, vote]) for vote in votes])
        else:
            votes = sum(results)
            proba = np.array([np.array([(1 - vote)/2, (1 + vote)/2]) for vote in votes])
        return proba

    def features_importance(self):
        """
        Compute features importances in estimators rules

        Returns:
        ----------
        importance : dict
            importances['avg'] : dict (feature id as key, mean importance as value)
                The mean importance of each feature over the estimators.
            importances['max'] : dict (feature id as key, max importance as value)
                The maximal importance of each feature over the estimators.
        """
        check_is_fitted(self, ["estimators", "estim_features"])
        importances = {'avg' : {}, 'max' : {}} # average and maximal feature/rule importances
        feature_id_occurences = {} # number of occurences of a feature in subsamples
        for (estim, features_idx) in zip(self.estimators, self.estim_features):
            # increment the total occurences of the feature :
            for id_feat in features_idx:
                if id_feat in feature_id_occurences:
                    feature_id_occurences[id_feat] += 1
                else:
                    feature_id_occurences[id_feat] = 1
            # sum the rules importances :
            #rules_importances = estim.get_rules_importances() # activate it when pyscm will implement importance
            rules_importances = np.ones(len(estim.model_.rules)) #delete it when pyscm will implement importance
            for rule, importance in zip(estim.model_.rules, rules_importances):
                global_feat_id = features_idx[rule.feature_idx]
                if global_feat_id in importances['avg']:
                    importances['avg'][global_feat_id] += importance
                    if importance > importances['max'][global_feat_id]:
                        importances['max'][global_feat_id] = importance
                else:
                    importances['avg'][global_feat_id] = importance
                    importances['max'][global_feat_id] = importance
        importances['avg'] = {k: round(v / feature_id_occurences[k], 3) for k, v in importances['avg'].items()}
        return importances

    def all_data_tiebreaker(self, model_type, feature_idx, thresholds, rule_type, X, y):
        """
        Choose a rule between rule with equal utility
        Select the one that have the best accuracy if applied alone on all the data

        Parameters :
        ----------
        model_type : strint ('conjunction' or 'dijunction')
            type of the model
        feature_idx, thresholds, rule_type : arrays
            description of the rules
        X, y : arrays
            a dataset used to compare rules

        Returns:
        ----------
        ID of the rule to select
        """
        keep_id = 0
        keep_id_score = -1
        for k in range(len(feature_idx)):
            feat_id, threshold, r_type = feature_idx[k], thresholds[k], rule_type[k]
            stump = DecisionStump(feature_idx=feat_id, threshold=threshold, kind=r_type)
            rule_classif = stump.classify(X).astype('int')
            rule_global_score = (rule_classif == y).sum()
            if rule_global_score > keep_id_score:
                keep_id = k
                keep_id_score = rule_global_score
        return keep_id

    def classifiers_disagreement(self, X):
        """
        Compute disagreement between estimators

        Returns:
        ----------
        disagreement : dict 
            disagreement['global'] : int
                global disagreement, computed with the following formula :
                average(disagreement(c1, c2))
                for (c1, c2) all the pairs of distinct estimators in the model
                with 
                disagreement(c1, c2) = (# of examples where c1 and c2 differs)/(# of examples)
            disagreement['estims'] : list
                list of disagreements of each estimator with the global model
            disagreement['heatmap'] : matrix
                heatmap of mutual disagreement for each pair of classifiers
        """
        check_is_fitted(self, ["estimators", "estim_features"])
        disagreement = {'global' : 0, 'estims' : []}
        total_comp = 0 # number of comparison of estimators
        global_pred = self.predict(X)
        heatmap = np.zeros((self.n_estimators, self.n_estimators))
        for kA in range(len(self.estimators)):
            estimA = self.estimators[kA]
            predA = estimA.predict(X[:, self.estim_features[kA]])
            disagree = np.not_equal(global_pred, predA).sum()/len(X) # disagreement of estimator A and global model
            disagreement['estims'].append(disagree)
            for kB in range(len(self.estimators)):
                if kB == kA:
                    continue # do not compare the estimator with itself
                estimB = self.estimators[kB]
                predB = estimB.predict(X[:, self.estim_features[kB]])
                disagree = np.not_equal(predB, predA).sum()/len(X) # disagreement of estimators A and B
                heatmap[kA][kB] = disagree
                disagreement['global'] += disagree
                total_comp += 1
        disagreement['global'] = disagreement['global']/total_comp
        disagreement['heatmap'] = heatmap
        return disagreement
    
    def get_estimators_indices(self):
        """
        Get drawn indices along both sample and feature axes
        """
        for seed in self._seeds:
            # operations accessing random_state must be performed identically to those in 'fit'
            feature_indices = sample_without_replacement(self._pop_features, self._max_features, random_state=seed)
            samples_indices = sample_without_replacement(self._pop_samples, self._max_samples, random_state=seed)
            yield samples_indices

    def score(self, X, y):
        check_is_fitted(self, ["estimators", "estim_features"])
        X, y = check_X_y(X, y)
        return accuracy_score(y, self.predict(X))

In [95]:
### Ici on instancie la classe RandomScmClassifier() et on vérifie les paramètres par défaut
result = RandomScmClassifier()
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(result.get_params())

Parameters currently in use:

{'max_features': 0.5,
 'max_rules': 10,
 'max_samples': 0.5,
 'min_cq_combination': False,
 'min_cq_mu': 0.01,
 'model_type': 'conjunction',
 'n_estimators': 100,
 'n_jobs': None,
 'p_options': [1.0],
 'random_state': None}


In [96]:
### Ici on divise les données avec un train/test de 80/20 et on rajoute un seed de 42
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [97]:
### Ici on entraine le modèle SCM avec les paramètres par défaut:
result.fit(X_train, y_train)

In [98]:
### Ici on calcule les prédictions avec le X_test puis on calcule la Test Accuracy
from sklearn.metrics import accuracy_score
from sklearn import metrics
predictions = result.predict(X_train)
print("Train Accuracy:", metrics.accuracy_score(y_train, predictions))

Train Accuracy: 0.7875


In [99]:
### Ici on calcule les prédictions avec le X_test puis on calcule la Test Accuracy
from sklearn.metrics import accuracy_score
from sklearn import metrics
predictions = result.predict(X_test)
print("Test Accuracy:", metrics.accuracy_score(y_test, predictions))

Test Accuracy: 0.8


In [100]:
### Ici, vérification avec f1_score
from sklearn.metrics import f1_score
f1_score(y_test, predictions)

0.6666666666666666

In [101]:
### Ici on reentraine le modèle avec du Boostrapping 60 fois
from sklearn.model_selection import train_test_split 
from sklearn import metrics
#### Bootstrapping ####
########################################################
# Creating empty list to hold accuracy values
AccuracyValues=[]
n_times=60
## Performing bootstrapping
from sklearn import metrics
from sklearn.model_selection import train_test_split 
for i in range(n_times):
    #Split the data into training and testing set
    from sklearn.model_selection import train_test_split
    # Changing the seed value for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+i)
    result = RandomScmClassifier(max_features = 0.5, max_samples = 0.5, model_type = 'disjunction', n_estimators = 10, p_options = [5.0])
    result.fit(X_train, y_train)
    ########################################################
    prediction = result.predict(X_test)
    Accuracy=metrics.accuracy_score(y_test, prediction)
    print(Accuracy)
    AccuracyValues.append((Accuracy))
    #print(Accuracy)
    print(AccuracyValues)
     #Measuring accuracy on Testing Data
#Accuracy=100- (np.mean(np.abs((y_test - prediction) / y_test)) * 100)
    
    # Storing accuracy values
#print(AccuracyValues.append(np.round(Accuracy)))
    
################################################
# Result of all bootstrapping trials
#print(AccuracyValues)
 
# Final accuracy
print('Final average accuracy',np.mean(AccuracyValues))

0.5
[0.5]
0.3
[0.5, 0.3]
0.55
[0.5, 0.3, 0.55]
0.45
[0.5, 0.3, 0.55, 0.45]
0.55
[0.5, 0.3, 0.55, 0.45, 0.55]
0.65
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65]
0.45
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45]
0.65
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65]
0.7
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7]
0.5
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7, 0.5]
0.35
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7, 0.5, 0.35]
0.6
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7, 0.5, 0.35, 0.6]
0.4
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7, 0.5, 0.35, 0.6, 0.4]
0.55
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7, 0.5, 0.35, 0.6, 0.4, 0.55]
0.6
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7, 0.5, 0.35, 0.6, 0.4, 0.55, 0.6]
0.75
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7, 0.5, 0.35, 0.6, 0.4, 0.55, 0.6, 0.75]
0.4
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.7, 0.5, 0.35, 0.6, 0.4, 0.55, 0.6, 0.75, 0.4]
0.6
[0.5, 0.3, 0.55, 0.45, 0.55, 0.65, 0.45, 0.65, 0.

In [102]:
### Ici on commence la Cross Validation pour trouver la meilleure combinaison de paramètres
from sklearn.model_selection import RandomizedSearchCV
# The number of base estimators in the ensemble.
n_estimators = [1, 10, 100, 1000] 
# Number of maximum features The number of features to draw from X to train each base estimator (without replacement.)
max_features = [0.25, 0.5, 0.75, 1.0] 
# Maximum samples
max_samples = [0.25, 0.5, 0.75, 1.0]
# p_options: The estimators will be fitted with values of p found in p_options 
p_options = [[0.1], [0.5], [1.0], [5.0], [10.0]]
# model_type: type of estimators to build
model_type = ['conjunction', 'disjunction']
# bootstrap
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_samples': max_samples,
               'p_options': p_options,
               'model_type': model_type}
pprint(random_grid)

{'max_features': [0.25, 0.5, 0.75, 1.0],
 'max_samples': [0.25, 0.5, 0.75, 1.0],
 'model_type': ['conjunction', 'disjunction'],
 'n_estimators': [1, 10, 100, 1000],
 'p_options': [[0.1], [0.5], [1.0], [5.0], [10.0]]}


In [ ]:
### Ici on entraîne les différentes combinaisons de paramètres
# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rf = RandomForestClassifier()
CVresult = RandomScmClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
CVresult = RandomizedSearchCV(estimator = CVresult, param_distributions = random_grid, n_iter = 200, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
CVresult.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 367 tasks      | elapsed:  3.5min


In [ ]:
CVresult.best_params_   ### Meilleurs paramètres obtenus en Cross-Validation

{'max_features': 1.0,
 'max_samples': 0.5,
 'model_type': 'disjunction',
 'n_estimators': 100,
 'p_options': [5.0]}

In [ ]:
### Nouvelle instantiation du RandomSCM avec les paramètres trouvés en Cross-Validation
CVresult = RandomScmClassifier(max_features = 1.0, max_samples = 0.5, model_type = 'disjunction', n_estimators = 100, p_options = [5.0])

In [ ]:
### entraînement du modèle avec les paramètres trouvés en Cross-Validation
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
CVresult.fit(X_train, y_train)

In [ ]:
### Calcul de y_test et de l'accuracy 
from sklearn.metrics import accuracy_score
from sklearn import metrics
predictions = CVresult.predict(X_test)
print("Test Accuracy:", metrics.accuracy_score(y_test, predictions))

Test Accuracy: 0.6


In [ ]:
### Vérification avec f1_score
from sklearn.metrics import f1_score
f1_score(y_test, predictions)

0.6363636363636364

In [ ]:
### Boostraping 60 fois avec Paramètres obtenus en Cross-Validation
from sklearn.model_selection import train_test_split 
from sklearn import metrics
#### Bootstrapping ####
########################################################
# Creating empty list to hold accuracy values
AccuracyValues=[]
n_times=60
## Performing bootstrapping
from sklearn import metrics
from sklearn.model_selection import train_test_split 
for i in range(n_times):
    #Split the data into training and testing set
    from sklearn.model_selection import train_test_split
    # Changing the seed value for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+i)
    CVresult = RandomScmClassifier(max_features = 1.0, max_samples = 0.5, model_type = 'disjunction', n_estimators = 100, p_options = [5.0])
    CVresult.fit(X_train, y_train)
    ########################################################
    prediction = CVresult.predict(X_test)
    Accuracy=metrics.accuracy_score(y_test, prediction)
    print(Accuracy)
    AccuracyValues.append((Accuracy))
    #print(Accuracy)
    print(AccuracyValues)
    
    #Creating the model on Training Data
    #DTree=RegModel.fit(X_train,y_train)
    #prediction=DTree.predict(X_test)
 
    #Measuring accuracy on Testing Data
#Accuracy=100- (np.mean(np.abs((y_test - prediction) / y_test)) * 100)
    
    # Storing accuracy values
#AccuracyValues.append(np.round(Accuracy))
    
################################################
# Result of all bootstrapping trials
print(AccuracyValues)
 
# Final accuracy
print('Final average accuracy',np.mean(AccuracyValues))
#print("Test Accuracy:", metrics.accuracy_score(y_test, y_final ))

0.6
[0.6]
0.6
[0.6, 0.6]
0.4
[0.6, 0.6, 0.4]
0.7
[0.6, 0.6, 0.4, 0.7]
0.6
[0.6, 0.6, 0.4, 0.7, 0.6]
0.7
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7]
0.6
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6]
0.5
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5]
0.6
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6]
0.55
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55]
0.35
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55, 0.35]
0.5
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55, 0.35, 0.5]
0.7
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55, 0.35, 0.5, 0.7]
0.6
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55, 0.35, 0.5, 0.7, 0.6]
0.75
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55, 0.35, 0.5, 0.7, 0.6, 0.75]
0.65
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55, 0.35, 0.5, 0.7, 0.6, 0.75, 0.65]
0.65
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55, 0.35, 0.5, 0.7, 0.6, 0.75, 0.65, 0.65]
0.5
[0.6, 0.6, 0.4, 0.7, 0.6, 0.7, 0.6, 0.5, 0.6, 0.55, 0.35, 0.5, 0.7, 0.6, 0.75, 0.65, 0.65, 0.5]
0.45
[0.6, 0.6, 0.4, 0.7